In [ ]:
# Import the necessary libraries
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup

In [ ]:
# Objective: Scrape comments from a given video.

# Running Selenium webdriver with target URL.
driver = webdriver.Chrome()
driver.get('https://www.youtube.com/watch?v=0JE5LrPBSb0')
driver.set_page_load_timeout(10)

In [ ]:
driver.get('https://www.youtube.com/watch?v=NOpmy7Kqgok')

In [ ]:
# Import page content. Make sure to scroll the page as much as needed before executing this command.
page_content = driver.page_source

In [ ]:
# Define a function that takes page content as input and outputs dataframe with scraped data.
def scrape_video(page_content, URL):
    
    # Parsing page content as BeautifulSoup object
    soup = BeautifulSoup(page_content, 'html.parser')
    
    # Create a BeautifulSoup ResultSet with data for all the posts to be scraped
    comments_raw = soup.find_all('ytd-comment-renderer', class_='style-scope ytd-comment-thread-renderer')
    
    # Create an empty list which would be appended with all comments
    comments = []
    
    # Iterate on the ResultSet to extract data for individual posts
    for comment in comments_raw:
        comment_data = {}
        
        # Getting the author
        author_element = comment.find('h3', class_='style-scope ytd-comment-renderer')
        if author_element:
            comment_data['Author'] = author_element.text.strip()
        else:
            'N/A'
            
        # Getting the Text
        text_element = comment.find('yt-formatted-string', class_='style-scope ytd-comment-renderer')
        if text_element:
            comment_data['Text'] = text_element.text
        else:
            'N/A'
            
        # Geting Timestamp
        time_element = comment.find('yt-formatted-string', class_='published-time-text style-scope ytd-comment-renderer')
        if time_element:
            comment_data['Timestamp'] = time_element.text
        else:
            'N/A'
        
        # Getting like count
        like_element = comment.find('span', class_='style-scope ytd-comment-action-buttons-renderer')
        if like_element:
            comment_data['No. of Likes'] = like_element.text.strip()
        else:
            'N/A'
            
        # URL
        comment_data['URL'] = URL
        
        # Appending the comment_data dict to the list
        comments.append(comment_data)
    
    # Create a DataFrame from the list of dictionaries after the loop
    df = pd.DataFrame(comments)
    return df

In [ ]:
df_temp = scrape_video(page_content, 'https://www.youtube.com/watch?v=NOpmy7Kqgok')

In [ ]:
df = pd.concat([df, df_temp], ignore_index=True)

In [ ]:
driver.quit()

In [ ]:
# Define a function that takes the dataframe and filename as input and writes an excel file to disk
def write_to_disk(df, filename):
    out_path = f'C:\\Users\\BINARY COMPUTERS\\Documents\\{filename}.xlsx'
    writer = pd.ExcelWriter(out_path , engine='xlsxwriter')
    df.to_excel(writer, sheet_name='Sheet1')
    writer.save()
    print(f"Write Complete. You can access the file at {out_path}")

In [ ]:
write_to_disk(df, 'GPT Store-YouTube')